# 📍 Notebook 03 — Análise Geográfica do Eleitorado de Vila Velha

Este notebook realiza a análise geográfica utilizando:
- Base tratada `base_vila_velha.csv`
- Shapefile dos bairros (`/dados/geodata/bairros_vila_velha.*`)
- Biblioteca `geopandas` para manipulação espacial
- Mapas temáticos sobre sexo, idade, escolaridade e participação por bairro


## 🔧 1. Importação de bibliotecas

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
import numpy as np

plt.style.use('seaborn-v0_8-whitegrid')

## 📁 2. Caminhos dos arquivos

In [ ]:
repo = 'analise-perfil-eleitor-vila-velha'

base_path = f"{repo}/dados/base_vila_velha.csv"
shape_path = f"{repo}/dados/geodata/bairros_vila_velha.shp"

## 📥 3. Carregamento da base de eleitores

In [ ]:
df = pd.read_csv(base_path, delimiter=',')
print(df.shape)
df.head()

## 🗺️ 4. Carregamento do shapefile dos bairros

In [ ]:
gdf_bairros = gpd.read_file(shape_path, encoding='utf-8')
gdf_bairros = gdf_bairros.to_crs(epsg=3857)
gdf_bairros.head()

## 🔄 5. Agregação de dados por bairro
Agrupamos a base para obter indicadores por bairro.

In [ ]:
# Correção do nome do bairro
df['bairro'] = df['bairro'].str.upper().str.strip()
gdf_bairros['nome_bairro'] = gdf_bairros['nome_bairro'].str.upper().str.strip()

# Agregação
agg = df.groupby('bairro').agg(
    total_eleitores=('titulo', 'count'),
    media_idade=('idade', 'mean'),
    perc_mulheres=('sexo', lambda x: (x=='FEMININO').mean()*100),
    perc_homens=('sexo', lambda x: (x=='MASCULINO').mean()*100)
).reset_index()

agg.head()

## 🔗 6. União espacial entre base e shapefile

In [ ]:
geo = gdf_bairros.merge(agg, left_on='nome_bairro', right_on='bairro', how='left')
geo.head()

# 🗺️ 7. Mapas Temáticos

## 🗺️ 7.1 Mapa de Total de Eleitores por Bairro

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
geo.plot(column='total_eleitores', cmap='Blues', legend=True, ax=ax)
ctx.add_basemap(ax, source=ctx.providers.Cartodb.Positron)
ax.set_title('Total de Eleitores por Bairro – Vila Velha', fontsize=16)
ax.axis('off')
plt.show()

## 🗺️ 7.2 Percentual de Mulheres por Bairro

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
geo.plot(column='perc_mulheres', cmap='Purples', legend=True, ax=ax)
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite)
ax.set_title('% de Mulheres por Bairro – Vila Velha', fontsize=16)
ax.axis('off')
plt.show()

## 🗺️ 7.3 Idade Média do Eleitor por Bairro

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
geo.plot(column='media_idade', cmap='OrRd', legend=True, ax=ax)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Voyager)
ax.set_title('Idade Média do Eleitor por Bairro – Vila Velha', fontsize=16)
ax.axis('off')
plt.show()

# 📌 8. Destaques e Insights Automáticos

In [ ]:
print('Top 5 bairros com mais eleitores:')
display(agg.nlargest(5, 'total_eleitores')[['bairro','total_eleitores']])

print('\nBairros com maior % de mulheres:')
display(agg.nlargest(5, 'perc_mulheres')[['bairro','perc_mulheres']])

print('\nBairros com maior idade média:')
display(agg.nlargest(5, 'media_idade')[['bairro','media_idade']])